# The Numerical Sandbox: Compression Experiment

**Romain Beucher**
Version 0.1

romain.beucher@unimelb.edu.au

The following notebook is an implementation of the Numerical Sandbox Compression Experiment similar to [Buiter et al., 2006](http://sp.lyellcollection.org/content/253/1/29). The test is commonly referred as one of the GEOMOD benchmarks and is used to test the large deformation viscous-plastic behaviour of geodynamic numerical codes.

The initial purpose of the GEOMOD numerical experiments was to compare results from analog and numerical experiments. It is not a numerical benchmark sensus-stricto.

In [ ]:
import UWGeodynamics as GEO
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, Image
from IPython.display import clear_output

In [ ]:
u = GEO.u

## Scaling

In [ ]:
import underworld as uw
import glucifer

In [ ]:
# define all widgets

fs = {'description_width': 'initial'} # full style
time = widgets.IntSlider(description='no. minutes ', min=1,max=30,style=fs, value=2)
wall_vx = widgets.BoundedFloatText(value=-2.5,min=-200.,max=200.0,
                                   description='Right wall velocity [cm/hr]', style=fs,disabled=False )
res = widgets.BoundedFloatText(value=0.2,min=0.02,max=1.0,
                               description='Initial Resolution [cm]', style=fs,disabled=False )
outputbox = widgets.Text(
    value='output/compression',
    description='output path:',
    disabled=False
)

w_button = widgets.Button(
    description='Do science',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

outputPath = './outputs_tutorial4'

def display_options():
    display(Image('../images/CompressionSetUp.png'))
    display(time)
    display(wall_vx)
    display(res)
    display(outputbox)
    display(w_button)

In [ ]:
display_options()

In [ ]:
velocity = 2.5 * u.centimeter / u.hour
model_length = 40. * u.centimeter
model_height = 10. * u.centimeter
refViscosity = 1e7 * u.pascal * u.second
bodyforce = 1560 * u.kilogram / u.metre**3 * 9.81 * u.meter / u.second**2

KL = model_length
Kt = KL / velocity
KM = bodyforce * KL**2 * Kt**2

GEO.scaling_coefficients["[length]"] = KL
GEO.scaling_coefficients["[time]"] = Kt
GEO.scaling_coefficients["[mass]"]= KM

# General parameters

In [ ]:
def make_model(ctx):
    # clear output then display widget again
    clear_output()
    display_options()
    
    global outputPath 
    outputPath = str(outputbox.value)
    import os
    outputPath = os.path.abspath(outputPath)+'/'
    
    import math
    nx = model_length.magnitude / res.value
    ny = model_height.magnitude / res.value
    
    nx = int(math.ceil(nx / 2.) * 2)
    ny = int(math.ceil(ny / 2.) * 2)
    
    Model = GEO.Model(elementRes=(nx,ny), 
                  minCoord=(0. * u.centimeter, -3.5 * u.centimeter), 
                  maxCoord=(40. * u.centimeter, 6.5 * u.centimeter))
    print nx, ny
    Model.outputDir=outputPath
    Model.minViscosity = 1.0e5 * u.pascal * u.second
    Model.maxViscosity = 1e12 * u.pascal * u.second
    Model.mesh_advector(axis=0)
    
    vx = GEO.nd(wall_vx.value * u.centimetre / u.hour)
    
    air        = Model.add_material(name="Air", shape=GEO.shapes.Layer(top=Model.top, bottom=0.0))
    sand1      = Model.add_material(name="Sand1", shape=GEO.shapes.Layer(top=air.bottom, bottom=Model.bottom))
    sand2      = Model.add_material(name="Sand2", shape=GEO.shapes.Layer(top=-1.0 * u.centimetre, bottom=-1.5*u.centimetre))
    microbeads = Model.add_material(name="Microbeads", shape=GEO.shapes.Layer(top=-2.5 * u.centimetre, bottom=-3.0*u.centimetre))

    import numpy as np

    wedge = [( 30.* u.centimeter, 0. * u.centimeter),
             (Model.maxCoord[0], 10.*u.centimeter * np.tan(np.radians(10))),
             (Model.maxCoord[0], 0. * u.centimeter)]

    sand3     = Model.add_material(name="Sand3", shape=GEO.shapes.Polygon(wedge))
    
    
    import numpy as np

    x = np.linspace(Model.minCoord[0], Model.maxCoord[0], 1000) * u.centimeter
    interface1 = GEO.nd(sand1.top)
    interface2 = GEO.nd(sand2.top)
    interface3 = GEO.nd(sand2.bottom)
    interface4 = GEO.nd(microbeads.top)
    interface5 = GEO.nd(microbeads.bottom)

    interface1 = Model.add_passive_tracers(name="Interface1", vertices=[x, interface1])
    interface2 = Model.add_passive_tracers(name="Interface2", vertices=[x, interface2])
    interface3 = Model.add_passive_tracers(name="Interface3", vertices=[x, interface3])
    interface4 = Model.add_passive_tracers(name="Interface4", vertices=[x, interface4])
    interface5 = Model.add_passive_tracers(name="Interface5", vertices=[x, interface5])
    
    air.density        = 10. * u.kilogram / u.metre**3
    sand1.density      = 1560. * u.kilogram / u.metre**3
    sand2.density      = 1560. * u.kilogram / u.metre**3
    sand3.density      = 1560. * u.kilogram / u.metre**3
    microbeads.density = 1480. * u.kilogram / u.metre**3
    
    air.viscosity        = 1.0e5 * u.pascal * u.second
    sand1.viscosity      = 1.0e12 * u.pascal * u.second
    sand2.viscosity      = 1.0e12 * u.pascal * u.second
    sand3.viscosity      = 1.0e12 * u.pascal * u.second
    microbeads.viscosity = 1.0e12 * u.pascal * u.second
    
    sandFriction = np.tan(np.radians(36.0))
    sandFrictionW = np.tan(np.radians(31.0))

    microbeadsFriction = np.tan(np.radians(22.0))
    microbeadsFrictionW = np.tan(np.radians(21.0))

    sand1.plasticity       = GEO.DruckerPrager(cohesion=10.*u.pascal, frictionCoefficient=sandFriction, frictionAfterSoftening=sandFrictionW)
    sand2.plasticity       = GEO.DruckerPrager(cohesion=10.*u.pascal, frictionCoefficient=sandFriction, frictionAfterSoftening=sandFrictionW)
    sand3.plasticity       = GEO.DruckerPrager(cohesion=10.*u.pascal, frictionCoefficient=sandFriction, frictionAfterSoftening=sandFrictionW)
    microbeads.plasticity  = GEO.DruckerPrager(cohesion=10.*u.pascal, frictionCoefficient=microbeadsFriction, frictionAfterSoftening=microbeadsFrictionW)
    conditions = [(Model.y < GEO.nd(Model.bottom +  0.2 * u.centimeter), 0.0),
              (Model.y > GEO.nd(Model.bottom +  0.4 * u.centimeter), vx),
              (True, (vx / GEO.nd(0.2 * u.centimeter) * (Model.y - GEO.nd(Model.bottom) - GEO.nd(0.2 * u.centimeter))))]
    
    Model.set_velocityBCs(left=[0.,0.], right=[conditions, 0.], bottom=[0.,0.])
    
    Model.set_frictional_boundary(left=True, right=True, bottom=True, top=False, friction=19.0, thickness=3)
    
    GEO.rcParams["solver"] = "mumps"
    GEO.rcParams["penalty"] = 1e6
    
    Model.run_for(time.value * u.minute, checkpoint_interval=0.5 * u.minute, glucifer_outputs=True )

# Initial Setup

# Model Geometry

The shortening experiment follows a more or less classic set-up used to study fold-and-thrust belt evolution with analogue models. Shortening is achieved by moving the right-hand wall inward while the left wall and base are held fixed. The model consists of ‘sand’ layers with an embedded 0.5 cm thick layer of weaker ‘microbeads'. Total height is 10 cm. The initial width of the model is 40 cm. At the right side an initial wedge 10 cm long with a 10 degrees surface slope overlies the model. This wedge slope is in the stable field for a sand wedge (as calculated for this set-up from Dahlen 1984; Zhao et al. 1986). The

<img src="./images/CompressionSetUp.png" width=800>

We use a uniform resolution of 0.2 cm (400 x 100 elements)


## Material distribution

The way we choose do define the initial set up is based on the distribution of the different materials (or phases).

In [ ]:
w_button.on_click(make_model)

In [ ]:
def vizualise(path=None):
    # if no path provided use the global value from the widget
    if path == None:
        global outputPath
        path = outputPath
    import os
    if not os.path.exists(path):
        print("Can't find ", path)
        return

    vis = glucifer.lavavu.Viewer(database=path+'/glucifer.gldb')
    vis.control.Panel()

    vis.control.TimeStepper()
    vis.control.ObjectList()
    vis.control.show()